In [2]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import warnings
warnings.filterwarnings("ignore")

region = "us-west-2"
config = Config(
    region_name=region,
    signature_version = "v4",
    retries={
        "max_attempts":3,
        "mode" : "standard",
    }
)
bedrock_rt = boto3.client("bedrock-runtime", config=config)

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs = {
    "max_tokens" : 4096,
    "temperature" : 0.0,
    "stop_sequences" : ["Human"],
}

llm = ChatBedrock(
    client = bedrock_rt,
    model_id = model_id,
    model_kwargs = model_kwargs,
)

In [2]:
import getpass
import os
os.environ['TAVILY_API_KEY']= getpass.getpass()

In [3]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
    FunctionMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name= "messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

In [9]:
import operator
from typing import Annotated,Sequence,TypedDict
from langgraph.graph import END,StateGraph,START

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],operator.add]
    next: str

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Summarizer","Reviewer"]
system_prompt = (
    """
    You are a supervisor tasked with managing the conversation between the following
    workers: {members}. Given the following the git diff recieved,
    respond with the worker to act next. Each worker will perform a task and respond
    with their results and status. When finished, respond with FINISH.
    """
)

options = ["FINISH"] + members

function_def = {
    "name" : "route",
    "description" : "Select the next role",
    "parameters": {
        "title" : "routeSchema",
        "type" : "object",
        "properties" : {
            "next" : {
                "title" : "Next",
                "anyOf" : [
                    {"enum" : options}
                ],
            }
        },
        "required" : ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder(variable_name = "messages"),
        ("system" , 
        "Given the conversation above, who should act next? Or should we finish? Select one of: {options}"
        ),
    ]
).partial(options = str(options),members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.with_structured_output(function_def)
    | JsonOutputFunctionsParser
)

In [13]:
import functools
from langchain_core.messages import AIMessage

def agent_node(state,agent,name):
    result = agent.invoke(state)

    if result.tool_calls:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages" : [result],
        "next" : name,
    }

summarizer_agent = create_agent(
    llm,
    [],
    system_message = "You should provide a summary to cover all the changes in the git diff provided.",
)
summarizer_node = functools.partial(agent_node,agent = summarizer_agent,name = "Summarizer")

review_agent = create_agent(
    llm,
    [],
    system_message = "Using the summary provided by the Summarizer and the git diff provide potential errors and code optimizations."
)
review_node = functools.partial(agent_node,agent = review_agent,name = "Reviewer")


In [14]:
workflow = StateGraph(AgentState)

workflow.add_node("Summarizer",summarizer_node)
workflow.add_node("Reviewer",review_node)
workflow.add_node("supervisor",supervisor_chain)

for member in members:
    workflow.add_edge(member,"supervisor")

conditional_map = {k : k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor",lambda x : x["next"],conditional_map)
workflow.add_edge(START,"supervisor")

senior_engineer = workflow.compile()

In [ ]:
from IPython.display import Image,display

display(Image(senior_engineer.get_graph(xray = True).draw_mermaid_png()))

In [ ]:
events = senior_engineer.stream(
    {
        "messages": [
            HumanMessage(
                content="""
                diff --git a/README.md b/README.md
index 28e5da8..68e3474 100644
--- a/README.md
+++ b/README.md
@@ -1,2 +1 @@
-# research-agent
-It is a multi-agent wodiff --git a/README.md b/README.md
index 28e5da8..68e3474 100644
--- a/README.md
+++ b/README.md
@@ -1,2 +1 @@
-# research-agent
-It is a multi-agent workflow that uses langgraph library.
+# research-agent
\ No newline at end of file
rkflow that uses langgraph library.
+# research-agent
\ No newline at end of file

                """
            )
        ],
    },
    {"recursion_limit": 10},
)
for s in events:
    print(s)
    print("----")